In [1]:
from pathlib import Path
import json

SRC_DIR  = Path(r"/mnt/c/Users/Legion/Documents/jimmy tran/Automated-invoice-processing-system/BIO-TAGGING WITH GPT/tnghia_data")
DEST_DIR = Path(r"/mnt/c/Users/Legion/Documents/jimmy tran/Automated-invoice-processing-system/BIO-TAGGING-RENAME/nghia_data")
DEST_DIR.mkdir(parents=True, exist_ok=True)

def update_id(obj, new_id):
    """
    Add/replace the 'id' field in a JSON object (dict).
    Silently skip if obj is not a dict.
    """
    if isinstance(obj, dict):
        obj["id"] = new_id

for src_path in SRC_DIR.glob("*.json*"):        # handles .json or .jsonl
    text = src_path.read_text(encoding="utf-8")

    # --- 1. Detect structure ----------------------------------------------
    try:                        # try full-file JSON first
        data = json.loads(text)
        structure = "json"
    except json.JSONDecodeError:
        # fall back to JSON-Lines
        data = [json.loads(line) for line in text.splitlines() if line.strip()]
        structure = "jsonl"

    # --- 2. Patch IDs ------------------------------------------------------
    stem = src_path.stem                # filename without extension
    if structure == "json":
        if isinstance(data, list):
            for obj in data:
                update_id(obj, stem)
        else:                           # single dict
            update_id(data, stem)
    else:                               # jsonl
        for obj in data:
            update_id(obj, stem)

    # --- 3. Write out ------------------------------------------------------
    dest_path = DEST_DIR / src_path.name
    if structure == "json":
        dest_path.write_text(
            json.dumps(data, ensure_ascii=False, indent=2),
            encoding="utf-8")
    else:                               # keep JSON-Lines one per line
        dest_path.write_text(
            "\n".join(json.dumps(obj, ensure_ascii=False) for obj in data),
            encoding="utf-8")

print("✅  ID fields synced to filenames →", DEST_DIR)


✅  ID fields synced to filenames → /mnt/c/Users/Legion/Documents/jimmy tran/Automated-invoice-processing-system/BIO-TAGGING-RENAME/nghia_data
